In [1]:
import pickle
import numpy as np

ddir = "../../tonemap/bf_data/Nair_and_Abraham_2010/"

fn = ddir + "all_gals.pickle"
all_gals = pickle.load(open(fn, "rb"))

all_gals = all_gals[1:] # Why the first galaxy image is NaN?

good_gids = np.array([gal['img_name'] for gal in all_gals])

from astrobf.utils.misc import load_Nair
cat_data = load_Nair(ddir + "catalog/table2.dat")
# pd dataframe

cat = cat_data[cat_data['ID'].isin(good_gids)]

tmo_params = {'b': 6.0,  'c': 3.96, 'dl': 9.22, 'dh': 2.45}


In [2]:
import matplotlib.pyplot as plt 

from PIL import Image
from typing import Any, Callable, Optional, Tuple
from torchvision.datasets.vision import VisionDataset
from functools import partial
from astrobf.tmo import Mantiuk_Seidel

class TonemapImageDataset(VisionDataset):
    def __init__(self, 
                 data_array, 
                 tmo,
                 labels: Optional = None, 
                 train: bool=True,
                 transform: Optional[Callable] = None,
                 target_transform: Optional[Callable] = None,):
        self._array = data_array
        self._good_gids = np.array([gal['img_name'] for gal in data_array])
        self.img_labels = labels
        self.transform = transform
        self.target_transform = target_transform
        self.tmo = tmo
        self._bad_tmo=False

    def _apply_tm(self, image):
        try:
            return self.tmo(image)
        except ZeroDivisionError:
            print("division by zero. Probably bad choice of TM parameters")
            self._bad_tmo=True
            return image

    def _to_8bit(self, image):
        """
        Normalize per image (or use global min max??)
        """

        image = (image - image.min())/image.ptp()
        image *= 255
        return image.astype('uint8')        
    
    def __len__(self) -> int:
        return len(self._array)
    
    def __getitem__(self, idx: int) -> Tuple[Any, Any]:
        """
        For super
        """
        image, _segmap, weight = self._array[idx]['data']
        image[~_segmap.astype(bool)] = 0#np.nan # Is it OK to have nan?
        image[image < 0] = 0

        image = self._to_8bit(self._apply_tm(image))
        image = Image.fromarray(image)
        label = self.img_labels[idx]
        
        if self.transform is not None:
            image = self.transform(image)
            
        return image, label


In [4]:
all_data = TonemapImageDataset(all_gals, partial(Mantiuk_Seidel, **tmo_params),
                                     labels=cat['TT'].to_numpy(),
                                     train=False, 
                                     transform=None)

In [8]:
cc[1]

ID      J000007.81m000226.09
TT                         1
area                   99.72
Name: 6294, dtype: object

In [9]:
import os

In [16]:
data_dir = './datasets/Nair/'
if not os.path.isdir(data_dir):
    os.mkdir(data_dir)

In [17]:
for (im, _), cc in zip(all_data, cat.iterrows()):
    gid = cc[1]["ID"]
    tt = cc[1]['TT']
    tdir = data_dir + f'{tt}/'
    if not os.path.isdir(tdir):
        os.mkdir(tdir)
        
    im.save(tdir+f'{gid}.png')

/home/hoseung/Work/astroBF/astrobf/tmo.py:268: RuntimeWarning: divide by zero encountered in log10
  lp = np.log10(lum) # L prime


In [19]:
ls datasets/Nair/

0/  1/  10/  11/  12/  -2/  2/  -3/  3/  4/  -5/  5/  6/  7/  8/  9/  99/


저장된 이미지 크기가 제각각이지만 상관없음, 어차피 crop할거라. 